# Tutorial 05, Part 1: Pandas DataFrames 
[The official project homepage](https://pandas.pydata.org)

## DataFrames

[Pandas quick start guide for DataFrames](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

* A DataFrame is a labeled data struture that can be thought of as a 2D extension of the Series objects that we discussed in the first part of the tutorial
* A DataFrame can accept many types of input, from a 2D ndarray, multiple Series, a dict of 1D arrays, another DataFrame, etc
* Like a Series, DataFrames contain data values and their labels. Because we're now dealing with a 2D structure, we call the row labels the index argument and the column labels the column argument. 
    * Like a Series, if you don't explicitly assign row and column labels, then they will be auto-generated (but not as useful as specifying the labels yourself!)

<div class="alert alert-info">
Much of what we learned about Series objects will generalize to DataFrames, so here we'll focus on some of key functionality that might not be obvious based on the previous tutorial.
</div>

<div class="alert alert-info">
One more quick note: if using an older version of Python (earlier than 3.6) and Pandas (earlier than 0.23) and you create a DataFrame from a dict without explicitly specifying column names, then the column names will be entered into the DataFrame based on lexical order
</div>

## Import libs

In [1]:
# standard numpy and matplotlib imports
import numpy as np
import matplotlib as plt

# for plotting in a separte window (not inline with notebook output)
# %matplotlib qt

# import a generic pandas object and also a few specific functions that we'll use
import pandas as pd 
from pandas import DataFrame, read_csv

# new - get and store current file path for file i/o later on in tutorial
import os
cwd = os.getcwd()

# also define the default font we'll use for figures. 
fig_font = {'fontname':'Arial', 'size':'20'}

## Make up a data set to demonstrate functionality, will import some real data later on
* Here we'll pretend that we did a unit recording experiment 
    * There are two stimulus conditions
    * And we are recording from 10 different neurons 

In [2]:
# seed random number generator so that we're all seeing the same thing in class
np.random.RandomState(0)

In [3]:
# index lables for our 10 neurons...see previous tutorial for more elegant ways of generating
# index labels, here we're just going to write them out
neuron_labels = ['Nrn0', 'Nrn1','Nrn2','Nrn3','Nrn4','Nrn5','Nrn6','Nrn7','Nrn8','Nrn9']  

In [4]:
# generate response in each neuron to stimulus 1...
resp1_hz = [14, 27, 62, 88, 45, 56, 75, 63, 33, 46]

In [5]:
# generate a response to stimulus 2...use random.randint just for practice/fun
min_resp = 0  # inclusive
max_resp = 90 # exclusive
resp2_hz = np.random.randint(min_resp, max_resp, len(resp1_hz))

## New - use 'zip' function to wrap up the data from each list into one list
[reference page for zip](https://www.w3schools.com/python/ref_func_zip.asp)

* Operates just like it sounds  - takes a set of iterators and groups them together into a single iterator with the 1st element in the resultant iterator comprised of the first element of each iterator 'zipped' together, then the second element from each iterator zipped together, etc. 
* Length of resulting iterator limited by the length of the shortest input iterator!

<div class="alert alert-warning">
Because the length of the resulting iterator is limited by length of shortest input iterator, you can sometimes not get an error if you try to zip together iterators with unequal lengths - this is fine if intenitonal, but if the unequal length was caused by a bug, then you may not find it when using zip!
</div>

In [20]:
neuron_data = list(zip(resp1_hz, resp2_hz))
print(neuron_data)

print('Grab one index to see the two response arrays zipped together:')
print(neuron_data[9])

[(14, 66), (27, 42), (62, 20), (88, 32), (45, 58), (56, 83), (75, 69), (63, 86), (33, 85), (46, 65)]
Grab one index to see the two response arrays zipped together:
(46, 65)


## Make a dataframe object to hold the contents of the data set
[DataFrame help page](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)

* Just like with the pd.Series call, you specify the data, index labels (row labels in this case)
* In addition to row labels, you can also specify column labels (with 'columns')
* Can also specify data type (default is inferred)
* Like pd.Series you can ask for an independent copy of the data (copy=True) or you will get a view by default (i.e. copy=False)

In [26]:
df = pd.DataFrame(data = neuron_data, index=neuron_labels, columns = ['resp1', 'resp2'])

# take a look at the output...
display(df)   # compare to print(df) - looks nicer with display thanks to iPython backend 

,resp1,resp2
Nrn0,14,66
Nrn1,27,42
Nrn2,62,20
Nrn3,88,32
Nrn4,45,58
Nrn5,56,83
Nrn6,75,69
Nrn7,63,86
Nrn8,33,85
Nrn9,46,65


## Sidebar...Important bit of info for avoiding a common mistake/problem
* Note that if you make a DataFrame out of a set of Series (e.g. a dict of Series), then the resulting DataFrame index labels will be the union of the index labels in each Series
* This can be confusing because the DataFrame will still be formed even if you have mismatching labels or even if you have two series of different sizes..
* Fortunately, the misaligned (or missing) values will be filled in with NaNs ('Not-a-Number') to serve as a placeholder for the misaligned or missing info
* Quick demo below before continuing on with our sample neuron data from above 

In [42]:
# make a set of two Series with unequal lengths stored in a dict, 
# with each Series having data and index labels

# Note that Series 1 has 4 elements, but Series 2 has 5 elements!

data_dict = {'dict0' : pd.Series(data = np.random.randn(4), index=['0','1','2','3']), 
            'dict1' : pd.Series(data = np.random.randint(0,5,5), index=['0','1','2','3','4'])}

# make a data frame
weird_df = pd.DataFrame(data_dict)

# take a look - notice that pd.DataFrame did not throw an error even though
# the input Series are different sizes...however, it did mark the missing value 
# with a NaN
display(weird_df)

,dict0,dict1
0,-1.109275,3
1,-1.472553,2
2,2.944128,1
3,0.724026,1
4,NaN,1


weird! you have NaNs in your data!


<div class = "alert alert-important">
Because of the above behavior, often good to frequently check for NaNs in your data to identify processing steps that might not have gone awry...(unless you are expecting NaNs as part of routine processing, in which case this might not be very helpful). 
Many ways to do this, but here is one way that works pretty well with identifying the presence of a NaN anywhere in the DataFrame (which is handy as often the DataFrames are too large to easily see in their entirety) 
</div>

## Saving data in a csv file

In [ ]:
 # lets save our header as well so that it doesn't think our first row is the header when we read the file back in
df.to_csv('spike_rates.csv',index=False,header=True)

In [ ]:
# use our current working directory to build a path to the file
print(cwd)
file_name = cwd + '/spike_rates.csv'
print(file_name)

df = pd.read_csv(file_name)
df

## Get a high-level summary of the data

In [ ]:
df.describe()

## Can also apply a set of more targeted analyses using the df object

* [Pandas doc for all functions](https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats)

In [ ]:
df.mean(axis=0)

## Making cooler DataFrame styles (and more useful...although that should take a backseat to coolness)
[Check here for a bunch of neat style options](https://pandas.pydata.org/pandas-docs/stable/style.html)
* Simple demo - can write custom functions that highlight specific aspects of your data - can be very useful for more clearly highlighting/communicating key points in the data within a notebook  

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]